In [1]:
from datetime import datetime
from collections import defaultdict

from vnpy_evo.trader.database import get_database, DB_TZ
from vnpy_evo.trader.constant import Interval
from vnpy_evo.trader.object import BarData
from vnpy_evo.trader.utility import extract_vt_symbol

db = get_database()

In [2]:
# Load data
vt_symbols = ["BTCUSDT.BINANCE", "ETHUSDT.BINANCE"]
vt_symbols = ["BTCUSDT.BINANCE"] #, "ETHUSDT.BINANCE"]

history: dict[datetime, dict[str, BarData]] = defaultdict(dict)

for vt_symbol in vt_symbols:
    symbol, exchange = extract_vt_symbol(vt_symbol)

    bars: list[BarData] = db.load_bar_data(
        symbol=symbol,
        exchange=exchange,
        interval=Interval.MINUTE,
        start=datetime(2024, 8, 1, tzinfo=DB_TZ),
        end=datetime(2024, 9, 10, tzinfo=DB_TZ)
    )

    for bar in bars:
        history[bar.datetime][vt_symbol] = bar
            

In [3]:
# Create table
from vnpy_novastrategy import DataTable

table = DataTable(
    vt_symbols=vt_symbols,
    size=5,
    interval=Interval.MINUTE,
    extra_fields=["active_volume", "active_turnover", "trade_count"]
)

table.add_feature("range", "high_price - low_price")


In [4]:
# Update bars into table    
for dt, bars in history.items():
    table.update_bars(bars)

In [ ]:
# Get latest dataframe
df = table.get_df()
df